In [ ]:
# import all packages
import os, platform
import subprocess
import shutil
import nltk
from nltk import *

This module already contains a /pdf for all IMF Article IV files for analysis. 
I proceed by reading /pdf to new txt folder named /tmp 

In [5]:
# PROCESS PDF TO TXT

# pdf files dir
input = os.getcwd() + '\\pdf\\'
# final dir after converted
output = os.getcwd() + '\\tmp\\'
# check/create if output folder exist
if not os.path.exists(output):
   os.makedirs(output)

# process pdf to txt file save in /tmp
# check platform system Windows/Macs
xpdfPath = ''
if platform.system() == 'Windows':
     xpdfPath = os.getcwd() + '\\xpdfbin-win-3.04\\bin64\\'
else:
     xpdfPath = os.getcwd() + '\\xpdfbin-mac-3.04\\bin64\\'

# store .pdf files from input
doc_list = [pdf for pdf in os.listdir(input) if pdf.endswith('.pdf')]
# iterate and process doc_list to txt file
for doc in doc_list:
    subprocess.call('pdftotext ' + input + doc + ' ' + output + doc.replace('.pdf', '.txt'), cwd=xpdfPath,
                        shell=True)


Once I have a /tmp with all txt files, I start strip out Countries' names from within 
the document to save them as file names to an actual /txt folder.

In [6]:
# SET FILE NAME TO COUNTRY--YEAR

# clear/retrieve txt file
txt_list = [txt for txt in os.listdir(output) if txt.endswith('.txt')]
newOutput = os.getcwd() + '\\txt\\'
# iterate and process txt_list to new txt folder
for f in txt_list:
    fp = open(output + f)
    fileName = ''
    docDate = ''
    for i, line in enumerate(fp):
       # retrieve doc date from 1st line
       if i == 0:
       # print(line.replace('\n', ''))
          docDate = line.replace('\n', '')
       # retrieve doc title from range(2,4)
       if i in range(2, 4) and 'IMF' not in line:
          # make sure line is not an empty line
          if line.strip():
             fileName = line.replace('\n', '') + '--' + docDate
       if i > 4:
          break
    fp.close()
    if not os.path.exists(newOutput):
       os.makedirs(newOutput)

    shutil.copy(output + f, newOutput + fileName + '.txt')

Remove the /tmp folder once copying process is done

In [7]:
shutil.rmtree(output)

One we have the /txt folder, we can pass the txt to our nlp training model. I will start with tokenizing the text.

In [51]:
# TOKENIZE WORDS FROM TXT FILE

from nltk import word_tokenize
from nltk.corpus import stopwords
import string

txtF = os.getcwd() + '/txt/ANGOLA--February 2017.txt'
txtF = open(txtF)

# continue to expand this regex for common stop signs in document
docStopSigns = ['\\n', '--', '\'s', '...']

# our stop list = English stopwords + common punctuation + docStopSigns
stop = stopwords.words('english') + [i for i in string.punctuation] + docStopSigns

# read txt file to string and loop word-to-word by word_tokenize
# read bagOfWords to list MINUS ones from the stop list
# finally, join list of string together making a sequences of string
text = ' '.join( w for w in word_tokenize(txtF.read()) if w not in stop)

In [70]:
# FIND 50 MOST COMMON WORDS

# find 50 most frequent words in the txt by FreqDist from nltk
tokens = word_tokenize(text)
nltkText = nltk.Text(tokens)


fd = FreqDist(nltkText)
common50 = fd.most_common(50)
sorted(common50)

[('0', 95),
 ('0.0', 116),
 ('1', 65),
 ('1.6', 48),
 ('10', 57),
 ('2', 67),
 ('20', 57),
 ('2014', 52),
 ('2015', 88),
 ('2016', 132),
 ('2017', 95),
 ('40', 53),
 ('ANGOLA', 65),
 ('Angola', 150),
 ('BNA', 68),
 ('Debt', 46),
 ('FUND', 61),
 ('GDP', 217),
 ('IMF', 49),
 ('INTERNATIONAL', 59),
 ('In', 52),
 ('MONETARY', 59),
 ('Oil', 58),
 ('Percent', 45),
 ('The', 168),
 ('U.S.', 88),
 ('authorities', 89),
 ('balance', 73),
 ('banks', 52),
 ('debt', 105),
 ('dollar', 46),
 ('dollars', 45),
 ('domestic', 58),
 ('exchange', 119),
 ('financial', 45),
 ('fiscal', 104),
 ('foreign', 45),
 ('gross', 46),
 ('growth', 93),
 ('market', 48),
 ('non-oil', 52),
 ('oil', 100),
 ('percent', 288),
 ('policy', 59),
 ('price', 56),
 ('public', 62),
 ('rate', 136),
 ('sector', 93),
 ('shock', 52),
 ('staff', 54)]

Since technical terms are usually complicated and containing multiple characters. I attempt to get rid of short words 

In [85]:
# check if word > 5 characters
# lowercase to prevent repeat count: exp: ANGOLA vs angola
longCharacter = [w.lower() for w in nltkText if len(w) > 5]

# Count words freq from new word list and choose most common 50
technicalCommon50 = sorted(FreqDist(longCharacter).most_common(50))
technicalCommon50

[('account', 44),
 ('angola', 215),
 ('angolan', 39),
 ('authorities', 95),
 ('average', 42),
 ('balance', 103),
 ('baseline', 44),
 ('capital', 39),
 ('central', 35),
 ('change', 43),
 ('credit', 30),
 ('currency', 48),
 ('current', 47),
 ('deficit', 30),
 ('dollar', 46),
 ('dollars', 45),
 ('domestic', 71),
 ('economic', 49),
 ('economy', 32),
 ('exchange', 142),
 ('exports', 45),
 ('external', 71),
 ('financial', 72),
 ('fiscal', 130),
 ('foreign', 68),
 ('framework', 34),
 ('government', 42),
 ('growth', 109),
 ('including', 40),
 ('increase', 44),
 ('inflation', 57),
 ('interest', 49),
 ('international', 121),
 ('investment', 31),
 ('market', 52),
 ('medium', 43),
 ('monetary', 125),
 ('non-oil', 93),
 ('percent', 333),
 ('period', 42),
 ('policy', 78),
 ('prices', 44),
 ('primary', 61),
 ('private', 35),
 ('projected', 37),
 ('public', 98),
 ('reserves', 44),
 ('revenue', 53),
 ('sector', 107),
 ('spending', 31)]

In [86]:
nltkText.similar('deficit')

balance adjustment revenue trade
